## Software Engineering for Probabilistic Programming
### using CmdstanPy and plotnine

Mitzi Morris 
Stan Development Team  
Columbia University, New York NY  
August 23, 2022  

In [ ]:
# import all libraries used in this notebook
import os
import numpy as np
import pandas as pd
from cmdstanpy import CmdStanModel

# plotting libs
import matplotlib.pyplot as plt
import plotnine as p9

# suppress plotnine warnings
import warnings
warnings.filterwarnings('ignore')

# setup plotnine look and feel
p9.theme_set(
  p9.theme_grey() + 
  p9.theme(text=p9.element_text(size=10),
        plot_title=p9.element_text(size=12),
        axis_title_x=p9.element_text(size=11),
        axis_title_y=p9.element_text(size=11),
        axis_text_x=p9.element_text(size=8),
        axis_text_y=p9.element_text(size=8)
       )
)
xlabels_90 = p9.theme(axis_text_x = p9.element_text(angle=90, hjust=1))


import logging
cmdstanpy_logger = logging.getLogger("cmdstanpy")
cmdstanpy_logger.disabled = True

mn_radon = pd.read_csv(os.path.join('data','mn_radon.csv'))
mn_counties = pd.read_csv(os.path.join('data','mn_uranium.csv'))

In [ ]:
mn_radon.drop(columns=['log_uranium']).iloc[[1,5,58]]

In [ ]:
mn_counties[:3]

In [ ]:
print(f'amount of data for Minnesota\n {len(mn_radon)} homes\n {len(mn_counties)} counties\n')
print(f'summary statistics for log_radon\n{mn_radon["log_radon"].describe()[1:]}')

In [ ]:
(p9.ggplot()
    + p9.geom_density(data=(mn_radon), mapping=p9.aes(x='log_radon'), color='darkblue')
    + p9.xlab("log radon levels") + p9.ylab("density")
    + p9.theme(figure_size=(5,2))
)

In [ ]:
(p9.ggplot()
    + p9.geom_density(data=(mn_radon), mapping=p9.aes(x='log_radon'), color='darkblue', alpha=0.2, linetype='dashed')
    + p9.geom_density(data=(mn_radon.loc[mn_radon['floor']==0]), mapping=p9.aes(x='log_radon'), color='darkorange')
    + p9.geom_density(data=(mn_radon.loc[mn_radon['floor']==1]), mapping=p9.aes(x='log_radon'), color='darkviolet')
    + p9.xlab("log radon levels") + p9.ylab("density")
    + p9.theme(figure_size=(5,2))
)

In [ ]:
# overlay histograms
(p9.ggplot(data=mn_radon, mapping=p9.aes(x='log_radon'))
    + p9.geom_histogram(data=(mn_radon.loc[mn_radon['floor']==0]), fill='orange', color='darkgreen', binwidth=0.2, alpha=0.7)
    + p9.geom_histogram(data=(mn_radon.loc[mn_radon['floor']==1]), fill='violet', color='blue', binwidth=0.2, alpha=0.6)
    + p9.xlab("log radon levels") + p9.ylab("number of observations")
    + p9.theme(figure_size=(5,2))
)

In [ ]:
pct_1 = round((mn_radon.floor.sum() / len(mn_radon) * 100))
pct_0= round(100 - pct_1)
print(f'floor 0: {pct_0}%\nfloor 1: {pct_1}%')

In [ ]:
print(f'Number of counties: {mn_radon.county.nunique()}')
print(f'Counties with measurements from floor 0: {mn_radon[mn_radon["floor"]==0]["county"].nunique()}')
print(f'Counties with measurements from floor 1: {mn_radon[mn_radon["floor"]==1]["county"].nunique()}')

In [ ]:
(p9.ggplot(data=mn_counties)
    + p9.geom_histogram(mapping=p9.aes(x='homes'), bins=40)
    + p9.geom_text(data=mn_counties[mn_counties['homes'] > 25],
                   mapping=p9.aes(x='homes', y=2, label='county'),
                   size=8, nudge_x=5, nudge_y=0.5)
    + p9.xlab("homes per county") + p9.ylab("")
    + p9.theme(figure_size=(12,4))
)

In [ ]:
(p9.ggplot(data=mn_counties, mapping=p9.aes(x='homes', y='log_uranium'))
    + p9.geom_point(fill='orange', color='darkred')
    + p9.geom_text(data=mn_counties[mn_counties['homes']>25],
                   mapping=p9.aes(label='county'),
                   size=8, nudge_x=4, nudge_y=0.1)
    + p9.xlab("homes per county") + p9.ylab("county soil log_uranium")
    + p9.theme(figure_size=(12,4))
)

In [ ]:
uranium_desc = mn_counties.sort_values(by='log_uranium', ascending=False).reset_index()

(p9.ggplot(data=mn_radon, mapping=p9.aes(x='county',y='log_radon'))
    + p9.geom_boxplot(width=2, varwidth=True, outlier_alpha=0.4)
    + p9.scale_x_discrete(limits=uranium_desc['county'], expand=(0,1))
    + p9.ggtitle("Counties ordered by soil uranium high (left) to low (right)")
    + p9.ylab("range of home radon measurements")
    + xlabels_90
    + p9.theme(figure_size=(12,3))
)

In [ ]:
# assemble data
radon_data = {"N": len(mn_radon), "x": mn_radon.floor.astype(float), "y": mn_radon.log_radon}
radon_data["J"] = 85
radon_data["county"] = mn_radon.county_id

# compile, fit models

no_pooling_model = CmdStanModel(stan_file=os.path.join('stan', 'radon_np.stan'))
no_pooling_fit = no_pooling_model.sample(data=radon_data)

no_pool_alpha = no_pooling_fit.stan_variable('alpha')
no_pool_alpha_mean = np.mean(no_pool_alpha, axis=0)  # axis=0 uses all rows, i.e., per-column mean
no_pool_alpha_lower = np.quantile(no_pool_alpha, 0.16, axis=0)
no_pool_alpha_upper = np.quantile(no_pool_alpha, 0.84, axis=0)
no_pool_alpha_pd = pd.DataFrame(data={
    "mean": no_pool_alpha_mean,
    "upper": no_pool_alpha_upper, 
    "lower": no_pool_alpha_lower, 
    "county":mn_counties['county']
})

# assemble dataframes for plotnine plots

partial_pooling_alpha_model = CmdStanModel(stan_file=os.path.join('stan', 'radon_pp_alpha.stan'))
partial_pooling_alpha_fit = partial_pooling_alpha_model.sample(data=radon_data)

part_pool_mu_alpha = partial_pooling_alpha_fit.stan_variable('mu_alpha').mean()
part_pool_alpha = partial_pooling_alpha_fit.stan_variable('alpha')
part_pool_alpha_mean = np.mean(part_pool_alpha, axis=0)
part_pool_alpha_lower = np.quantile(part_pool_alpha, 0.16, axis=0)
part_pool_alpha_upper = np.quantile(part_pool_alpha, 0.84, axis=0)
part_pool_alpha_pd = pd.DataFrame(
    data={
        "mean": part_pool_alpha_mean,
        "upper": part_pool_alpha_upper, 
        "lower": part_pool_alpha_lower, 
        "county":mn_counties['county']
    }
)

# x axis ordering
pop_desc = mn_counties.sort_values(by='homes', ascending=False).reset_index()


(p9.ggplot(data=part_pool_alpha_pd)
 + p9.geom_segment(
     mapping=p9.aes(x='county', xend='county', y='lower', yend='upper'),
     size=1.7, color='blue', alpha=0.7,
 )
 + p9.geom_segment(data=no_pool_alpha_pd,
     mapping=p9.aes(x='county', xend='county', y='lower', yend='upper'),
     size=1.4, color='orange', alpha=0.6,
 )
 + p9.geom_point(mapping=p9.aes(x='county', y='mean'))
 + p9.geom_hline(yintercept=part_pool_mu_alpha, color='darkblue', size=1)
 + p9.scale_x_discrete(limits=pop_desc['county']) + p9.scales.ylim(0,3.5)
 + p9.ggtitle("multilevel varying intercept model estimates for alpha (basement log_radon level)")
 + p9.xlab("ordered by observations per county, desc") + p9.ylab("central 67% interval")
 + xlabels_90
 + p9.theme(figure_size=(12,2.5)) 
)

In [ ]:
# posterior predictive check - median estimated value

y_rep_pp = partial_pooling_alpha_fit.draws_pd(vars='y_rep')

# for each of the 85 counties, estimate the median y_rep
stat_median = []
for i in range(1,86):
    idxs = mn_radon.index[mn_radon['county_id'] == i].tolist()
    stat_median.append(np.median(y_rep_pp.iloc[:, idxs].to_numpy().flatten()))

(p9.ggplot()
  + p9.geom_boxplot(data=mn_radon,
                    mapping=p9.aes(x='county',y='log_radon'), color='orange', fatten=2, alpha=0.7, outlier_alpha=0.3)
  + p9.geom_point(mapping=p9.aes(x=mn_counties.county, y=stat_median), color='purple')
  + xlabels_90
  + p9.theme(figure_size=(12,2.5))
)

In [ ]:
# posterior density plot - get a random sample of replicates

sz = 100
y_rep_pp = partial_pooling_alpha_fit.draws_pd(vars='y_rep')

# each column is a replicate of the data, using estimates of alpha, beta
y_rep_pp_sample = y_rep_pp.sample(sz).reset_index(drop=True).T

# plot actual distribution of the data against predicted new data
pp_ppc = p9.ggplot()
for i in range(sz):
    pp_ppc = pp_ppc + p9.geom_density(mapping=p9.aes(x=y_rep_pp_sample[i]), color='lightblue', alpha=0.4)
pp_ppc = (pp_ppc 
          + p9.geom_density(data=mn_radon, mapping=p9.aes(x='log_radon'), color='darkblue', size=1.1)
          + p9.xlab("log radon levels") + p9.ylab("density")
          + p9.theme(figure_size=(6,2))
         )
pp_ppc